In [1]:
import numpy as np
from scipy.misc import derivative
import matplotlib.pyplot as plt
from numba import njit

def modelFunction(x,y):
   return (0.5*x + 1.5*y)**4

@njit(cache=True)
def randomMultivariateNormalSVD(ave, cov, n=None):
    if n is None:
        n=1
    X = np.random.randn(n, len(ave))
    U, s, V =  np.linalg.svd(cov, full_matrices=False)
    S = np.diag ( np.sqrt(s) )
    A = U.dot(S)
    Y = ave + np.dot(X,A.T)

    return Y


In [2]:
xave = 1.0
ux = 0.5
yave = 4.0
uy = 2.0
rho = 0.5

ave = np.array( [xave, yave] )
cov = np.array([[ux**2, ux*uy*rho], [uy*ux*rho, uy**2]])

In [3]:
# Sandwich formula
eps = 1e-5
dzdx = (modelFunction(xave*(1+eps), yave) - modelFunction(xave*(1-eps), yave))/(2*eps*xave)
dzdy = (modelFunction(xave, yave*(1+eps)) - modelFunction(xave, yave*(1-eps)))/(2*eps*yave)
S = np.array([dzdx, dzdy]).reshape( (1, len(ave)) )
zave = modelFunction(xave, yave) 
uz = np.sqrt( (S.dot(cov)).dot(S.T) )[0,0]
print(zave, uz)

1785.0625 3441.041387431365


In [4]:
# UT sampling
r = np.linalg.matrix_rank(cov)
n=2*r+1
kappa = 3.0 - r


U, s, V =  np.linalg.svd(cov, full_matrices=False)
A = U.dot( np.diag ( np.sqrt(s) ) )

X = np.zeros((n, len(ave)))
weight = np.zeros(n)
factor = np.sqrt(r+kappa)
X[0,:] = ave
for i in range(r):
    ip = 2*i+1
    im = 2*i+2
        
    X[ip,:] = ave + factor*A[:,i]
    X[im,:] = ave - factor*A[:,i]
    weight[ip]   = 1/(2*(r+kappa))
    weight[im]   = 1/(2*(r+kappa))
weight[0] = 1.0 - weight.sum()

"""
aveUT = np.zeros( len(ave) )
for i in range(n):
    aveUT += weight[i] *(X[i,:])

covUT = np.zeros_like(cov)
for i in range(n):
    sigma = (X[i,:] - aveUT).reshape((1,len(ave)))
    covUT += weight[i] * sigma.T.dot(sigma)
print(covUT)
"""

z = modelFunction(X[:,0], X[:,1])
zave = 0.0
for i in range(n):
    zave += weight[i]*z[i]

uz = 0.0
for i in range(n):
    sigma = (z[i] - zave)
    uz += weight[i] * sigma**2
uz = np.sqrt(uz)
print(zave, uz)

4560.405261281118 7024.116977820551


In [5]:
# random sampling
xy = randomMultivariateNormalSVD(ave, cov, 1000000)

z = modelFunction(xy[:,0], xy[:,1])
zave = z.mean()
uz = z.std(ddof=1)
print(zave, uz)

4555.080394713339 7528.703893805509
